# Evaluating XLSR-53 and XLS-R fine-tuned models
*by Dragoș Alexandru Bălan*

The script used to test the models I experimented with can be found here.

**IMPORTANT:** The experiments have been conducted in a Linux environment. I have no knowledge on how the notebook works on Windows or MacOS, so no support is provided for those operating systems.

### Dependencies
First, the dependencies required to run this notebook are found below. I recommend to first create and activate a virtual environment, then run the commands below in that environment. You will only need to run it once.

In [ ]:
!pip install datasets
!pip install transformers
!pip install torchaudio
!pip install jiwer
!pip install soundfile
!pip install librosa
!pip install evaluate

### DISCLAIMER

Run all the cells below until the 'Running evaluation' section in order to load the test dataset and preprocess it (and load additional dependencies).

In [1]:
from transformers import pipeline
from datasets import load_dataset
import evaluate

#### Logging into Huggingface

In case you don't have the dataset cached already, you will need to login to Huggingface so an authentication token can be stored locally. First, you will need to register for an account here: https://huggingface.co/join

After creating an account, you will need an access token in order to login to Huggingface on this notebook. You can create one by going to the profile picture icon on the top-right of the website -> Settings -> Access Tokens. In here, click on 'New token'. Input any name you want but **make sure to set the role to 'write'**.

Once your access token is generated, run the cell below, copy the access token, and paste it when requested.

In [2]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /home4/s3944867/.cache/huggingface/token
Login successful


Now, we load the test split of the Frisian subset of Common Voice 8.0 from Huggingface

In [3]:
test_set = load_dataset('mozilla-foundation/common_voice_8_0', 'fy-NL', split='test', use_auth_token=True)

Found cached dataset common_voice_8_0 (/home4/s3944867/.cache/huggingface/datasets/mozilla-foundation___common_voice_8_0/fy-NL/8.0.0/b2f8b72f8f30b2e98c41ccf855954d9e35a5fa498c43332df198534ff9797a4a)


Same preprocessing as during training must be applied in order to get accurate results

In [4]:
import re
chars_to_remove_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\']'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_remove_regex, '', batch["sentence"]).lower()
    return batch

test_set = test_set.map(remove_special_characters)

Loading cached processed dataset at /home4/s3944867/.cache/huggingface/datasets/mozilla-foundation___common_voice_8_0/fy-NL/8.0.0/b2f8b72f8f30b2e98c41ccf855954d9e35a5fa498c43332df198534ff9797a4a/cache-868b569a23578be9.arrow


## Running evaluation

It will take around 4 hours to run the evaluation. The cell below can be adapted for other models/datasets as well. The metric can also be changed to "cer" if you wish to evaluate the Character Error Rate (CER) of the model.

To change the model to be evaluated, make sure to change the 'model' attribute of the 'pipeline()' function.

**NOTE:** not all the experiments are present in this notebook.

### Evaluation of experiment 3 (10 hours of train data)

In [5]:
transcriber = pipeline("automatic-speech-recognition", model='greenw0lf/wav2vec2-large-xls-r-1b-frisian-cv-8-10h')
wer = evaluate.load("wer")

labels = []
preds = []
i = 0

for recording in test_set:
    label = labels.append(recording['sentence'])
    pred = preds.append(transcriber(recording['audio'])['text'])
    i += 1
    print(str(i) + '/' + str(len(test_set)))
    print('predicted: ' + preds[i-1])
    print('actual: ' + labels[i-1])
    
print(wer.compute(predictions=preds, references=labels))

1/3024
predicted: der sieten gjin oare minsken by de bestjoerder yn de auto
actual: der sieten gjin oare minsken by de bestjoerder yn de auto
2/3024
predicted: ik fyn it wol spitich datst it net even sjongst
actual: ik fyn it wol spitich datst it net even sjongst
3/3024
predicted: douwe wurket yn de soarch
actual: douwe wurket yn e soarch
4/3024
predicted: de swiere noardwestewyn hold lang oan
actual: de swiere noardwestewyn hold lang oan
5/3024
predicted: hy lijt ûnder de druk en ferwachtingen fan syn famylje
actual: hy lijt ûnder de druk en ferwachtingen fan syn famylje
6/3024
predicted: de gemeente moat mear ynvestearje om de ynwenners oan hin baan te helpen
actual: de gemeente moat mear ynvestearje om de ynwenners oan in baan te helpen
7/3024
predicted: tsjin twa oere fan e middei wie de steuring frijwal oeral foarby
actual: tsjin twa oere fan e middei wie de steuring frijwol oeral foarby
8/3024
predicted: wy beseagen dêr ek in heel lyts mar moai museum
actual: wy beseagen dêr ek i

68/3024
predicted: do hiest tink ik nea ferwachte datst sa heech yn it klassemint komme soest
actual: do hiest tink ik nea ferwachte datst sa heech yn it klassemint komme soest
69/3024
predicted: mei dit systeem betellet men foar it jinge dêrt men nei sjocht
actual: mei dit systeem betellet men foar itjinge dêrt men nei sjocht
70/3024
predicted: op moandeitemiddei is der in fyts tocht ûnder lieding fan in gids
actual: op moandeitemiddei is der in fytstocht ûnder lieding fan in gids
71/3024
predicted: wat docksto
actual: wat dochsto
72/3024
predicted: elke plant hat in minimum oan rûmte nedich
actual: elke plant hat in minimum oan rûmte nedich
73/3024
predicted: it jongste is der bymial ôf
actual: it jongste is der by my al ôf
74/3024
predicted: ik wit noch wol dat de earste trekkers kamen
actual: ik wit noch wol dat de earste trekkers kamen
75/3024
predicted: dat hinget ôf fan bedriuwsgrutte sektor kultuer fan e ûndernimming en bedriuwsstrategy
actual: dat hinget ôf fan bedriuwsgrutte 

131/3024
predicted: hoe seist komkommer yn it spaansk
actual: hoe seist komkommer yn it spaansk
132/3024
predicted: masterwurken út lettere iuwen siere it renovearre museum
actual: masterwurken út lettere iuwen siere it renovearre museum
133/3024
predicted: har iene frije middei mei foar neat of nimmen wike
actual: har iene frije middei mei foar neat of nimmen wike
134/3024
predicted: dy binne fergees
actual: dy binne fergees
135/3024
predicted: sjoch alle dagen ek by de deiaksje foar oanbidingen tsjin boaienprizen
actual: sjoch alle dagen ek by de deiaksje foar oanbiedingen tsjin boaiemprizen
136/3024
predicted: alle minsken gongen werom nei hûs deiene ei de oare
actual: alle minsken gongen werom nei hûs de iene nei de oare
137/3024
predicted: ha se no foar dizze eksposyzje ek neiwurk makke
actual: ha se no foar dizze eksposysje ek nij wurk makke
138/3024
predicted: sneontemoarn om tsien oere geanen de doarren fan syn tydlike winkel iepen
actual: sneontemoarn om tsien oere giene de do

196/3024
predicted: it waard in gesellige jûn dyt ôfsletten waard mei in hapke en in drankje
actual: it waard in gesellige jûn dyt ôfsletten waard mei in hapke en in drankje
197/3024
predicted: kom echter my oan jonges dan lit ik jim de tún sjen
actual: kom achter my oan jonges dan lit ik jim de tún sjen
198/3024
predicted: nim myn mei nei dyn nachtklup al wie it mar ien kear
actual: nim my mei nei dy nachtklup al wie it mear ien kear
199/3024
predicted: lykas by alles jielt hjir ek earst tinke dan dwaan
actual: lykas by alles jildt hjir ek earst tinke dan dwaan
200/3024
predicted: hja heart net dat wy deroankomme oant we deun foar har stean
actual: hja heart net dat wy der oankomme oant wy deun foar har steane
201/3024
predicted: lykas beppe bodde ek ús mem op it lân
actual: lykas beppe bodde ek ús mem op it lân
202/3024
predicted: it duorre net lang en de rest wie der ek al
actual: it duorre net lang en de rest wie der ek al
203/3024
predicted: guon boeren hawwe de pleats ferkocht
ac

261/3024
predicted: hy wit fan fuotbaljen allinne dat de bal rûn is
actual: hy wit fan fuotbaljen allinne dat de bal rûn is
262/3024
predicted: troch dy sitte wy yn dit parket
actual: troch dy sitte wy yn dit parket
263/3024
predicted: de keamers hawwe ekstra lege dakruten foar gasten yn in rolstoel
actual: de keamers hawwe ekstra lege dakruten foar gasten yn in rolstoel
264/3024
predicted: de ôfrûne dagen hiene se him ûnder druk set
actual: de ôfrûne dagen hiene se him ûnder druk set
265/3024
predicted: noch mar fjouwer nachtsjes sliepe en dan is it safier
actual: noch mar fjouwer nachtsjes sliepe en dan is it safier
266/3024
predicted: de direksje fan it sikehús wol net reagearje
actual: de direksje fan it sikehûs wol net reagearje
267/3024
predicted: fideo efterkant
actual: fideo efterkant
268/3024
predicted: hy fuottelt fan har de hakken út e skuûn
actual: hy fottelt foar har de hakken út e skuon
269/3024
predicted: let yn e nacht strampele it duo it sikehús yn
actual: let yn e nac

...

2737/3024
predicted: tap
actual: tab
2738/3024
predicted: ut de priiswinners fan elke kategory wurdt de deikampioen keazen
actual: ut e priiswinners fan elke kategory wurdt de deikampioen keazen
2739/3024
predicted: bring yn it sin woansdei om fjouwer oere middeisôfspraak by de kapper
actual: bring yn t sin woansdei om fjouwer oere middeis ôfspraak by de kapper
2740/3024
predicted: de delegaasje wol de problemen út it eigen doarp op it aljemint bringe
actual: de delegaasje wol de problemen út it eigen doarp op it aljemint bringe
2741/3024
predicted: wurk fan syn hân waard oankocht troch it ryk
actual: wurk fan syn hân waard oankocht troch it ryk
2742/3024
predicted: hja sei in pear kwarte wurden tsjin de massa yn de hau
actual: hja sei in pear koarte wurden tsjin de massa yn de hal
2743/3024
predicted: yn de eagen fan de rjochter binne alle saken like wichtich
actual: yn de eagen fan de rjochter binne alle saken like wichtich
2744/3024
predicted: der kaam in grutter ferlet fan iepenlof

2801/3024
predicted: de tún mei palmen e orchideeën sjocht der paradyslik út yn e moarnssinne
actual: de tún mei palmen en orchideeën sjocht der paradyslik út yn e moarnssinne
2802/3024
predicted: pas nei fyftjin jier socht se help
actual: pas nei fyftjin jier socht se help
2803/3024
predicted: it soe earst útstjoerd wurde yn petearfoarm
actual: it soe earst útstjoerd wurde yn petearfoarm
2804/3024
predicted: hja sjogge mei grutte eagen nei wat hjir bart
actual: hja sjogge mei grutte eagen nei wat hjir bart
2805/3024
predicted: ik ha hjir natuerlik wol t ien en oar meimakke
actual: ik ha hjir natuerlik wol it ien en oar meimakke
2806/3024
predicted: bist net lulk om wat ik sein ha
actual: bist net lulk om wat ik sein ha
2807/3024
predicted: ha jo fierder noch fragen
actual: hawwe jo fierder noch fragen
2808/3024
predicted: ik bin ferkeard sitearre en ik wol der fierder neat oer sizze
actual: ik bin ferkeard sitearre en ik wol der fierder neat oer sizze
2809/3024
predicted: de plysje do

2865/3024
predicted: sjoch op ús webside foar mear ynformaasje oer ús produkten
actual: sjoch op ús webside foar mear ynformaasje oer ús produkten
2866/3024
predicted: as men mei jins twaen reizget dielt men fansels tegearre in keamer
actual: as men mei jins twaen reizget dielt men fansels tegearre in keamer
2867/3024
predicted: ferline jier makken wy in rûnreis troch frankryk
actual: ferline jier makken wy in rûnreis troch frankryk
2868/3024
predicted: ik hoech net nei dat konsert
actual: ik hoech net nei dat konsert
2869/3024
predicted: ik ha in kear by har west en doe hat hja deroer ferteld
actual: ik ha in kear by har west en doe hat hja deroer ferteld
2870/3024
predicted: dit is it bonkerak fan in mûs
actual: dit is it bonkerak fan in mûs
2871/3024
predicted: ik siet yn gedacht en wei figueren te tekenjen
actual: ik siet yn e gedachten wei figueren te tekenjen
2872/3024
predicted: nije probleemgefallen wurde besprutsen en der wurdt besjoen wat de meast gaadlike help is
actual: nij

2926/3024
predicted: it is noch altyd net persyis bekend wat der dy freedtenachts bard is
actual: it is noch altyd net presys bekend wat der dy freedtenachts bard is
2927/3024
predicted: autoferhierbedriuw
actual: autoferhierbedriuw
2928/3024
predicted: de lien binne ûndjip mei frijrêstich wetter
actual: de leien binne ûndjip mei frij rêstich wetter
2929/3024
predicted: aginda besjen
actual: aginda besjen
2930/3024
predicted: hja wist it kampioenskip yn de dressuer op har namme te setten
actual: hja wist it kampioenskip yn de dressuer op har namme te setten
2931/3024
predicted: wiskje alles yn dizze rige
actual: wiskje alles yn dizze rige
2932/3024
predicted: de keamer stie blau fan de reek
actual: de keamer stie blau fan e reek
2933/3024
predicted: dat is de reden wêromt ik foarearst by him bliuw
actual: dat is de reden wêromt ik foarearst by him bliuw
2934/3024
predicted: ik bin op alle fokdagen yn fryslân
actual: ik bin op alle fokdagen yn fryslân
2935/3024
predicted: hy is woansdei

2991/3024
predicted: ik wachte en wachte oant ik my wreke kinne soe
actual: ik wachte en wachte oant ik my wreke kinne soe
2992/3024
predicted: begelieden wer iepenje
actual: begelieden wer iepenje
2993/3024
predicted: der wie noch in plakje frij neist in âlde frou mei in feloranje mûtske
actual: der wie noch in plakje frij neist in âlde frou mei in feloranje mûtske
2994/3024
predicted: se hat faak opnomen west yn in psychiatrysk sikehús
actual: se hat faak opnommen west yn in psychiatrysk sikehús
2995/3024
predicted: ja ik bin it der folslein mei iens hear
actual: ja ik bin it der folslein mei iens hear
2996/3024
predicted: wat hat dizze wurksume man in ûnbidige wurk ferset
actual: wat hat dizze wurksume man in ûnbidigen wurk ferset
2997/3024
predicted: skiermûntseach is in eilân fan rêst en romte
actual: skiermûntseach is in eilân fan rêst en romte
2998/3024
predicted: anke en yvonne rinne al jierren tegearre de saldeikuiertocht
actual: anke en yvonne rinne al jierren tegearre deseld

### Evaluation of experiment 4 (1 hour of train data)

In [6]:
transcriber = pipeline("automatic-speech-recognition", model='greenw0lf/wav2vec2-large-xls-r-1b-frisian-cv-8-1h')
wer = evaluate.load("wer")

labels = []
preds = []
i = 0

for recording in test_set:
    label = labels.append(recording['sentence'])
    pred = preds.append(transcriber(recording['audio'])['text'])
    i += 1
    print(str(i) + '/' + str(len(test_set)))
    print('predicted: ' + preds[i-1])
    print('actual: ' + labels[i-1])
    
print(wer.compute(predictions=preds, references=labels))

1/3024
predicted: der sieten gjin oare minsken by de bestjoerder yn de auto
actual: der sieten gjin oare minsken by de bestjoerder yn de auto
2/3024
predicted: ik fyn it wolspytich datdit net even sjongst
actual: ik fyn it wol spitich datst it net even sjongst
3/3024
predicted: dauwe wurket yn dei soarich
actual: douwe wurket yn e soarch
4/3024
predicted: de swier en noadwestewyn holt lang oan
actual: de swiere noardwestewyn hold lang oan
5/3024
predicted: hy leit ûnder de druk en ferwachtingen fan syn famylje
actual: hy lijt ûnder de druk en ferwachtingen fan syn famylje
6/3024
predicted: de geminte moat mear ynvestearje om de ynwenners oan in baan te helpen
actual: de gemeente moat mear ynvestearje om de ynwenners oan in baan te helpen
7/3024
predicted: tsjin twa oere fan e middei wie de steuring frij wol oeral foarby
actual: tsjin twa oere fan e middei wie de steuring frijwol oeral foarby
8/3024
predicted: wy beseagen dêr ek in heec lyts mar moai museum
actual: wy beseagen dêr ek in

68/3024
predicted: do hiest tink ik nea ferwachte dat sa heech yn it plassemint komme soest
actual: do hiest tink ik nea ferwachte datst sa heech yn it klassemint komme soest
69/3024
predicted: mei dit systeem betellet men foar it jinge dêrt men nei sjocht
actual: mei dit systeem betellet men foar itjinge dêrt men nei sjocht
70/3024
predicted: op moandeitemiddei is der in fytstocht oan de liening fan in gits
actual: op moandeitemiddei is der in fytstocht ûnder lieding fan in gids
71/3024
predicted: wat doksto
actual: wat dochsto
72/3024
predicted: elke plant hat de minimem oan rûmte nedich
actual: elke plant hat in minimum oan rûmte nedich
73/3024
predicted: it jongste is der bemil ôf
actual: it jongste is der by my al ôf
74/3024
predicted: ik wit noch wol dat de earste trek is kaamen
actual: ik wit noch wol dat de earste trekkers kamen
75/3024
predicted: dat hinget ôf fan bedriuwskritte sektor kultuer fan e ûndernimming en bebriuwstrategy
actual: dat hinget ôf fan bedriuwsgrutte sekto

131/3024
predicted: hoe seist kom kommer yn it spaansk
actual: hoe seist komkommer yn it spaansk
132/3024
predicted: masterwurken út lettere ioen siere it renovearre museum
actual: masterwurken út lettere iuwen siere it renovearre museum
133/3024
predicted: har iene frije middei mei foar neat of nimmen wike
actual: har iene frije middei mei foar neat of nimmen wike
134/3024
predicted: dy binne foar gee
actual: dy binne fergees
135/3024
predicted: sjoch alle dagen ek by de dijeaksje foar oanbitingen tsjin boaiemprizen
actual: sjoch alle dagen ek by de deiaksje foar oanbiedingen tsjin boaiemprizen
136/3024
predicted: alle minsken gong werom nei hûs en diene nei de oare
actual: alle minsken gongen werom nei hûs de iene nei de oare
137/3024
predicted: ha se no foar dizze ek persyzje ek neij wurk makke
actual: ha se no foar dizze eksposysje ek nij wurk makke
138/3024
predicted: sneontemoarn om tsien oere geane de doarren fan syn tiidlike winkel iepen
actual: sneontemoarn om tsien oere giene

196/3024
predicted: dwa hrden gesellige jûn dyt ôfsletten waard mei in hapke en in drankje
actual: it waard in gesellige jûn dyt ôfsletten waard mei in hapke en in drankje
197/3024
predicted: kom achter my oan jonges dan lit ik jim de tún sjen
actual: kom achter my oan jonges dan lit ik jim de tún sjen
198/3024
predicted: nim myn mârn e dy nachtklup al wie it mar ien kear
actual: nim my mei nei dy nachtklup al wie it mear ien kear
199/3024
predicted: lykas by alles hielt hjir ik earst tinke daan dwaan
actual: lykas by alles jildt hjir ek earst tinke dan dwaan
200/3024
predicted: hja heard net dat wy der oankomme oant it deun foar har stean
actual: hja heart net dat wy der oankomme oant wy deun foar har steane
201/3024
predicted: lykas beppe borde ek ús mem op it lân
actual: lykas beppe bodde ek ús mem op it lân
202/3024
predicted: it doarre net lang en de rest wie der ekk a
actual: it duorre net lang en de rest wie der ek al
203/3024
predicted: guon boeren hawwe de pleats ferkocht
actu

261/3024
predicted: hy wit fan fuortbaljen alinneh dat bol rûn is
actual: hy wit fan fuotbaljen allinne dat de bal rûn is
262/3024
predicted: troch dy sitte wy yn dit parket
actual: troch dy sitte wy yn dit parket
263/3024
predicted: de keamers hawwe ekstra leege dakruten foar gasten yn in roltstoel
actual: de keamers hawwe ekstra lege dakruten foar gasten yn in rolstoel
264/3024
predicted: de ôfrûne dagen hiene se him ûnder drukset
actual: de ôfrûne dagen hiene se him ûnder druk set
265/3024
predicted: noch mei fjouwer nachtsje sliepe en dan is it safier
actual: noch mar fjouwer nachtsjes sliepe en dan is it safier
266/3024
predicted: de direksje fan it sikehûs wol net reagearje
actual: de direksje fan it sikehûs wol net reagearje
267/3024
predicted: fideo efterkant
actual: fideo efterkant
268/3024
predicted: hy fottelt fan har de hakken út e skoûn
actual: hy fottelt foar har de hakken út e skuon
269/3024
predicted: let yn e nacht strampele de duo it sikehús yn
actual: let yn e nacht 

...

2732/3024
predicted: hawwe jo sûker en molke yn e koefje
actual: hawwe jo sûker en molke yn e kofje
2733/3024
predicted: ik sjoch de frou alle snein twa kear yn tsjerke
actual: ik sjoch de frou alle sneinen twa kear yn tsjerke
2734/3024
predicted: ik ha hieltyd ûen polityk dien sûnder dersels betterfan wurde te wollen
actual: ik ha hieltyd oan polityk dien sûnder der sels better fan wurde te wollen
2735/3024
predicted: oan de fruchten kin men de wiem
actual: oan e fruchten ken men de beam
2736/3024
predicted: kin ik de fyts hjir iene kwit
actual: kin ik de fyts hjir earne kwyt
2737/3024
predicted: tap
actual: tab
2738/3024
predicted: út de priiswinners fan elke kategory wurdt de deikampe joan keazen
actual: ut e priiswinners fan elke kategory wurdt de deikampioen keazen
2739/3024
predicted: bring yn it sin wenzje om fjouwer oere middeisôfspraak by de kapper
actual: bring yn t sin woansdei om fjouwer oere middeis ôfspraak by de kapper
2740/3024
predicted: de delegaasje wol de probleem ú

2796/3024
predicted: de wurdt al owen oer praat en dêr reitsje we noait oar útpraat
actual: dêr wurdt al iuwen oer praat en dêr reitsje we noait oer útpraat
2797/3024
predicted: nei twa oeren riden hie ik in lytse tritich kilometer ôflein
actual: nei twa oeren riden hie ik in lytse tritich kilometer ôflein
2798/3024
predicted: hy sit op it jild as de duvel op e siele
actual: hy sit op it jild as de duvel op e siele
2799/3024
predicted: navigearje nei haans
actual: navigearje nei harns
2800/3024
predicted: de ynsittenen fan de autos binne mei de skrik frijkomen
actual: de ynsittenden fan de autos binne mei de skrik frijkommen
2801/3024
predicted: de tún mei palmen en archideen sjocht de paradyslik út yn e moarnsinne
actual: de tún mei palmen en orchideeën sjocht der paradyslik út yn e moarnssinne
2802/3024
predicted: past nei fyftjin jier socht se help
actual: pas nei fyftjin jier socht se help
2803/3024
predicted: it soe earst útstjoerd wurde yn petier foarm
actual: it soe earst útstjo

2860/3024
predicted: sa op it each is der net folle foar oare
actual: sa op it each is der net folle feroare
2861/3024
predicted: de haadwegen tusken de grutte stêden binne frij fan tal
actual: de haadwegen tusken de grutte stêden binne frij fan tol
2862/3024
predicted: set do de flagge mar op
actual: set do de flagge mar op
2863/3024
predicted: wêr soe dat wy betellje moatte
actual: wêr soe k dat wei betelje moatte
2864/3024
predicted: yn it earsten fan it lân beliuwt it oan e jûn ta jûndeldrûch
actual: yn it easten fan it lân bliuwt it oan e jûn ta hielendal drûch
2865/3024
predicted: sjoch op ús website foar mear ynformaasje oer ús produkten
actual: sjoch op ús webside foar mear ynformaasje oer ús produkten
2866/3024
predicted: as men mei jins twagen reizight dielt men fan sels te heare in keamer
actual: as men mei jins twaen reizget dielt men fansels tegearre in keamer
2867/3024
predicted: ferline jier makken wie in rûnreis troch frankryk
actual: ferline jier makken wy in rûnreis t

2922/3024
predicted: der moat jild ynvestearre wurde yn saken as wurde gelegenheid en natoer
actual: der moat jild ynvestearre wurde yn saken as wurkgelegenheid en natoer
2923/3024
predicted: myn nacht mei milena hie de langsten nei dane net foerdreon
actual: myn nacht mei milena hie de langstme nei dana net ferdreaun
2924/3024
predicted: de baggaazje waard opslein yn in romte oan e ezingewei
actual: de bagaazje waard opslein yn in romte oan de ezingewei
2925/3024
predicted: yn it wykein en op moandei en tiisdei bin ik altyd thús
actual: yn it wykein en op moandei en tiisdei bin ik altyd thús
2926/3024
predicted: it is noch altyd net prsys bekend wat er dy freede nachtsbard is
actual: it is noch altyd net presys bekend wat der dy freedtenachts bard is
2927/3024
predicted: autoferhierbedrieuw
actual: autoferhierbedriuw
2928/3024
predicted: de lijen binne ûndiep mei frij rêstich wetter
actual: de leien binne ûndjip mei frij rêstich wetter
2929/3024
predicted: agen da besjen
actual: agind

2986/3024
predicted: ik hie oan de holsta kâlde fuorten
actual: ik hie oan de hals ta kâlde fuotten
2987/3024
predicted: hoe let begint it earste oere
actual: hoe let begjint it earste oere
2988/3024
predicted: ik kin der net om neitsje
actual: ik kin der net om laitsje
2989/3024
predicted: ik ha leard dat de minsken de rûn te jaan moat
actual: ik ha leard dat men minsken de rûmte jaan moat
2990/3024
predicted: der is in wurkgroep oprjochte foar de takamst fan de âldereins soarich
actual: der is in wurkgroep oprjochte foar de takomst fan de âldereinsoarch
2991/3024
predicted: ik wachte en wachte oant ik my wreeke kinne soe
actual: ik wachte en wachte oant ik my wreke kinne soe
2992/3024
predicted: buge leaden wer iepenje
actual: begelieden wer iepenje
2993/3024
predicted: der wie noch in plakje frij neist in âlde frou mei in fel oranje mûtske
actual: der wie noch in plakje frij neist in âlde frou mei in feloranje mûtske
2994/3024
predicted: se hat faak op nam west yn in psicheatrysk si

### Evaluation of experiment 5 (10 minutes of train data)

In [5]:
transcriber = pipeline("automatic-speech-recognition", model='greenw0lf/wav2vec2-large-xls-r-1b-frisian-cv-8-10m')
wer = evaluate.load("wer")

labels = []
preds = []
i = 0

for recording in test_set:
    label = labels.append(recording['sentence'])
    pred = preds.append(transcriber(recording['audio'])['text'])
    i += 1
    print(str(i) + '/' + str(len(test_set)))
    print('predicted: ' + preds[i-1])
    print('actual: ' + labels[i-1])
    
print(wer.compute(predictions=preds, references=labels))

1/3024
predicted: der syi it ein giein oreminsken by de wbestjuoeder yn de alto
actual: der sieten gjin oare minsken by de bestjoerder yn de auto
2/3024
predicted: ik fyn it dwolspitich dat s dit net ewen sjongst
actual: ik fyn it wol spitich datst it net even sjongst
3/3024
predicted: dauwe wurket yn di soaricht
actual: douwe wurket yn e soarch
4/3024
predicted: de swiere nord weste wyn holt lang om
actual: de swiere noardwestewyn hold lang oan
5/3024
predicted: hie leit ûnder de druk en ferwachtingen fansyn familje
actual: hy lijt ûnder de druk en ferwachtingen fan syn famylje
6/3024
predicted: de gemente moat mear yn festerje om de ynwenners oan in ban te helpen
actual: de gemeente moat mear ynvestearje om de ynwenners oan in baan te helpen
7/3024
predicted: tsjin tuwa oere fan de mije wie de steuring frij woal oerol farb
actual: tsjin twa oere fan e middei wie de steuring frijwol oeral foarby
8/3024
predicted: wy beseagen dr ik in hee lytst mar moei múseejem
actual: wy beseagen dêr

69/3024
predicted: mei dizy ste jyn betellet men foari gjiger dd mei nei sjo
actual: mei dit systeem betellet men foar itjinge dêrt men nei sjocht
70/3024
predicted: op mandje te middje is der in fytstocht ûnderliding fan in gits
actual: op moandeitemiddei is der in fytstocht ûnder lieding fan in gids
71/3024
predicted: woat dok sto
actual: wat dochsto
72/3024
predicted: elke plant ha de mynimm on rûmte nedif
actual: elke plant hat in minimum oan rûmte nedich
73/3024
predicted: dat jongkster is der bemiaou
actual: it jongste is der by my al ôf
74/3024
predicted: ik wit noch wol wat de easte trekers kamen
actual: ik wit noch wol dat de earste trekkers kamen
75/3024
predicted: dat hinge  fam betriuskrutte sektoer toetuer fan de ûnerniming  en bebliustrafiley
actual: dat hinget ôf fan bedriuwsgrutte sektor kultuer fan e ûndernimming en bedriuwsstrategy
76/3024
predicted: syn awekad oerde rjocht saak efte sletten duoarden boehe mine
actual: syn abbekaat woe de rjochtsaak efter sletten doar

133/3024
predicted: har iiene frije mitje mei foart neat of nimmen wiker
actual: har iene frije middei mei foar neat of nimmen wike
134/3024
predicted: dy binne ferges
actual: dy binne fergees
135/3024
predicted: sjoch alledagen ek by de die aksje for mbidigen tsin boajen frizen
actual: sjoch alle dagen ek by de deiaksje foar oanbiedingen tsjin boaiemprizen
136/3024
predicted: ale misken gongwerom nei s e djine eit de ore
actual: alle minsken gongen werom nei hûs de iene nei de oare
137/3024
predicted: has nou foar dize ek pesyzje ekg nei wurk mage
actual: ha se no foar dizze eksposysje ek nij wurk makke
138/3024
predicted: sneu de mon omt sien oere gin de duoarden fan syn tilike wenkel jipen
actual: sneontemoarn om tsien oere giene de doarren fan syn tydlike winkel iepen
139/3024
predicted: dize wurksjouws binne tider galik foerjo
actual: dizze workshops binne tige gaadlik foar jo
140/3024
predicted: bautse
actual: pauze
141/3024
predicted: it ûndersyk wur dientroch stúdenten dy yne e

198/3024
predicted: nimy moein dea dy nacht klip al wie it merynjinkeer
actual: nim my mei nei dy nachtklup al wie it mear ien kear
199/3024
predicted: lik as beosjieldjier ik eaastinke dan duwan
actual: lykas by alles jildt hjir ek earst tinke dan dwaan
200/3024
predicted: ja jerd net dat wy dre ûnkome ont wit dun foa hak steen
actual: hja heart net dat wy der oankomme oant wy deun foar har steane
201/3024
predicted: lyk as bepe bode ek ús meme op itt len
actual: lykas beppe bodde ek ús mem op it lân
202/3024
predicted: it duore net lang en de rest widreko
actual: it duorre net lang en de rest wie der ek al
203/3024
predicted: wom booeren hawwe de pleds ferkocht
actual: guon boeren hawwe de pleats ferkocht
204/3024
predicted: de dyk wurd breder se dat ryne takomst hûnderd kilemei te riden wurde kem
actual: de dyk wurdt breder sadat der yn de takomst hûndert kilometer riden wurde kin
205/3024
predicted: as it te hurdwid git i dyi ip m loft spil net troch
actual: as it te hurd waait gie

265/3024
predicted: noch ma fjoauwer nachtsje sliepeen dan is it sa fier
actual: noch mar fjouwer nachtsjes sliepe en dan is it safier
266/3024
predicted: de dyireksje fan it syke hs wol netre je gerje
actual: de direksje fan it sikehûs wol net reagearje
267/3024
predicted: fydio efster knt
actual: fideo efterkant
268/3024
predicted: hy fottel tfan har de haken út de skom
actual: hy fottelt foar har de hakken út e skuon
269/3024
predicted: let yn e nacht strampele it duo it syken húsyn
actual: let yn e nacht strampele it duo it sikehús yn
270/3024
predicted: mag hyi hat dy briif lezen
actual: mar hy hat dy brief lêzen
271/3024
predicted: om de prys hoefer jo i net te liten
actual: om e priis hoege jo it net te litten
272/3024
predicted: difoarsitter sjouke de han krige blomm en in boekebom
actual: deifoarsitter sjoukje de haan krige blommen en in boekebon
273/3024
predicted: de plysje koer far komme dat i ta in konfrantsje kam
actual: de plysje koe foarkomme dat it ta in konfrontaasje 

...

2719/3024
predicted: op merklik binne de wurde wurden son de reken kamer
actual: opmerklik binne de hurde wurden fan de rekkenkeamer
2720/3024
predicted: bistjer liear krest of in lierling
actual: bist hjir in learkrêft of in learling
2721/3024
predicted: dar is in dútdlik fskiltusten gehone stinns foeroringen en in depresje
actual: der is in dúdlik ferskil tusken gewoane stimmingsferoaringen en in depresje
2722/3024
predicted: langom let bi  ine sliep fan en jie nûfer greamt
actual: lang om let wie hja yn e sliep fallen en hie nuver dreamd
2723/3024
predicted: fan de achtsjyn klups wieine trii jitsjen en fúftjen for
actual: fan de achttjin klups wiene trije tsjin en fyftjin foar
2724/3024
predicted: it basanten ûndersyk dat de polisje mein dy hold hat sien tifsol smyiten
actual: it passsanteûndersyk dat de polysje moandei hold hat tsien tips opsmiten
2725/3024
predicted: is fantesyy git mei jús ûne hl sa oeremes as wy binne
actual: us fantasy giet mei ús oan e haal sa oerémis as wy bi

2782/3024
predicted: hy is noch oan it op knappem nei in knibel op eras
actual: hy is noch oan it opknappen nei in knibbeloperaasje
2783/3024
predicted: der wie no  restnt fan moans frizens yn e loft
actual: der wie noch in restant fan moarnsfrissens yn e loft
2784/3024
predicted: yn it midden fan e achtsjen dye i winne yn de fryslmnen dy grafurne
actual: yn de midden fan de achttjinde iuw wenne yn de fryslannen in dykgraaf
2785/3024
predicted: midde binne jo frou hofstra
actual: middei binne jo frou hofstra
2786/3024
predicted: de rigo skrose
actual: de rigel skrasse
2787/3024
predicted: de brnwacht is noh oan it nei bleisen
actual: de brânwacht is noch oan it neiblussen
2788/3024
predicted: ik ferfilmi jiren der kom net ynenmear o besyte
actual: ik ferfeel my hjir en der komt net ien mear op besite
2789/3024
predicted: w hie dat efter arjen soft
actual: wa hie dat efter arjen socht
2790/3024
predicted: nei in oantol mearejieren ferwachtete rginysje dite jier wer fol op besikes
actual

2848/3024
predicted: tsjin ien fan der beamen si it in swoarte spjocht mei fjooerejer de krun
actual: tsjin ien fan e beammen siet in swarte spjocht mei in fjoerreade krún
2849/3024
predicted: yn de betide nacht fersnuen op snein moste polysje yngripe by fjochter
actual: yn e betide nacht fan sneon op snein moast de plysje yngripe by fjochterij
2850/3024
predicted: de polysje krige to thedene soad meligen oerfjourwurk oger least
actual: de polysje kriget op t heden in soad meldingen oer fjurwurkoerlêst
2851/3024
predicted: wurdt sûnder haad letters
actual: wurd sûnder haadletters
2852/3024
predicted: hy is in kop gruter as sy siuster
actual: hy is in kop grutter as syn suster
2853/3024
predicted: al betid op e middei komte joûnspits op gon
actual: al betiid op e middei komt de jûnsspits op gong
2854/3024
predicted: meinskún út hiel frysno wi kam in towe wiken lang nei ljoauwert
actual: minsken út hiel fryslân wei kamen tolve wiken lang nei ljouwert
2855/3024
predicted: ja wardin oan hl

2911/3024
predicted: de boarge mafster docht in oprop om net sa gou lulikk te wurden
actual: de boargemaster docht in oprop om net sa gau lulk te wurden
2912/3024
predicted: nei allegedachten is de atetroditsjoger yn it fochteler ferjen bedare
actual: nei alle gedachten is de otter troch de tsjonger yn it fochtelerfean bedarre
2913/3024
predicted: man en frou binne net gelyk me wol likwierdich
actual: man en frou binne net gelyk mar wol lykweardich
2914/3024
predicted: aft is i  dobene forfjouwer
actual: acht is it dûbele fan fjouwer
2915/3024
predicted: nodr wol it net o mei duwam
actual: no dêr wol ik net oan meidwaan
2916/3024
predicted: it drûkte dr nei woare miten
actual: it rûkte dêr nei waarmiten
2917/3024
predicted: warneater my út noge lieater myitroch in taksje op hele
actual: wanneart er my útnûge liet er my troch in taksy ophelje
2918/3024
predicted: ja hie al heel faak bearen tsjoen moarûntanta nee en grisli
actual: hja hie al heel faak bearen sjoen mar oant dan ta nea in 

2976/3024
predicted: yn e kursis bedriuwspsyghology stiet de meinnske yn e organisaasje sintral
actual: yn e kursus bedriuwspsychology stiet de minske yn e organisaasje sintraal
2977/3024
predicted: aldeize gringene bleaden wurde rd yn de jest
actual: al dizze griene blêden wurde read yn e hjerst
2978/3024
predicted: ferpleeadste aliynniea
actual: ferpleats de alinea
2979/3024
predicted: has to ek in hús dier
actual: hasto ek in húsdier
2980/3024
predicted: ale hûnden binne trau
actual: alle hûnen binne trou
2981/3024
predicted: it leste ieal jier fam yn eineikse amen haw ik by harden ynne hús winne
actual: it lêste healjier foar myn eineksamen haw ik by harren yn e hús wenne
2982/3024
predicted: disze twardichters plate oer op fiering en it enfekte fam op it lettere liben
actual: dizze twa dichters prate oer opfieding en it effekt dêrfan op it lettere libben
2983/3024
predicted: ik win wurich en de kree gjeare noel koartol op syn frag
actual: ik wie wurch en ik reagearre nochal koartô

### Reproduction of the baseline results (XLSR-53 trained on train split of CV 8)

In [9]:
transcriber = pipeline("automatic-speech-recognition", model='wietsedv/wav2vec2-large-xlsr-53-frisian')
wer = evaluate.load("wer")

labels = []
preds = []
i = 0

for recording in test_set:
    label = labels.append(recording['sentence'])
    pred = preds.append(transcriber(recording['audio'])['text'])
    i += 1
    print(str(i) + '/' + str(len(test_set)))
    print('predicted: ' + preds[i-1])
    print('actual: ' + labels[i-1])
    
print(wer.compute(predictions=preds, references=labels))

/scratch/s3944867/xlsr_venv/lib/python3.9/site-packages/transformers/configuration_utils.py:379: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


1/3024
2/3024
3/3024
4/3024
5/3024
6/3024
7/3024
8/3024
9/3024
10/3024
11/3024
12/3024
13/3024
14/3024
15/3024
16/3024
17/3024
18/3024
19/3024
20/3024
21/3024
22/3024
23/3024
24/3024
25/3024
26/3024
27/3024
28/3024
29/3024
30/3024
31/3024
32/3024
33/3024
34/3024
35/3024
36/3024
37/3024
38/3024
39/3024
40/3024
41/3024
42/3024
43/3024
44/3024
45/3024
46/3024
47/3024
48/3024
49/3024
50/3024
51/3024
52/3024
53/3024
54/3024
55/3024
56/3024
57/3024
58/3024
59/3024
60/3024
61/3024
62/3024
63/3024
64/3024
65/3024
66/3024
67/3024
68/3024
69/3024
70/3024
71/3024
72/3024
73/3024
74/3024
75/3024
76/3024
77/3024
78/3024
79/3024
80/3024
81/3024
82/3024
83/3024
84/3024
85/3024
86/3024
87/3024
88/3024
89/3024
90/3024
91/3024
92/3024
93/3024
94/3024
95/3024
96/3024
97/3024
98/3024
99/3024
100/3024
101/3024
102/3024
103/3024
104/3024
105/3024
106/3024
107/3024
108/3024
109/3024
110/3024
111/3024
112/3024
113/3024
114/3024
115/3024
116/3024
117/3024
118/3024
119/3024
120/3024
121/3024
122/3024
123/3024
1

924/3024
925/3024
926/3024
927/3024
928/3024
929/3024
930/3024
931/3024
932/3024
933/3024
934/3024
935/3024
936/3024
937/3024
938/3024
939/3024
940/3024
941/3024
942/3024
943/3024
944/3024
945/3024
946/3024
947/3024
948/3024
949/3024
950/3024
951/3024
952/3024
953/3024
954/3024
955/3024
956/3024
957/3024
958/3024
959/3024
960/3024
961/3024
962/3024
963/3024
964/3024
965/3024
966/3024
967/3024
968/3024
969/3024
970/3024
971/3024
972/3024
973/3024
974/3024
975/3024
976/3024
977/3024
978/3024
979/3024
980/3024
981/3024
982/3024
983/3024
984/3024
985/3024
986/3024
987/3024
988/3024
989/3024
990/3024
991/3024
992/3024
993/3024
994/3024
995/3024
996/3024
997/3024
998/3024
999/3024
1000/3024
1001/3024
1002/3024
1003/3024
1004/3024
1005/3024
1006/3024
1007/3024
1008/3024
1009/3024
1010/3024
1011/3024
1012/3024
1013/3024
1014/3024
1015/3024
1016/3024
1017/3024
1018/3024
1019/3024
1020/3024
1021/3024
1022/3024
1023/3024
1024/3024
1025/3024
1026/3024
1027/3024
1028/3024
1029/3024
1030/3024
1031/3

1751/3024
1752/3024
1753/3024
1754/3024
1755/3024
1756/3024
1757/3024
1758/3024
1759/3024
1760/3024
1761/3024
1762/3024
1763/3024
1764/3024
1765/3024
1766/3024
1767/3024
1768/3024
1769/3024
1770/3024
1771/3024
1772/3024
1773/3024
1774/3024
1775/3024
1776/3024
1777/3024
1778/3024
1779/3024
1780/3024
1781/3024
1782/3024
1783/3024
1784/3024
1785/3024
1786/3024
1787/3024
1788/3024
1789/3024
1790/3024
1791/3024
1792/3024
1793/3024
1794/3024
1795/3024
1796/3024
1797/3024
1798/3024
1799/3024
1800/3024
1801/3024
1802/3024
1803/3024
1804/3024
1805/3024
1806/3024
1807/3024
1808/3024
1809/3024
1810/3024
1811/3024
1812/3024
1813/3024
1814/3024
1815/3024
1816/3024
1817/3024
1818/3024
1819/3024
1820/3024
1821/3024
1822/3024
1823/3024
1824/3024
1825/3024
1826/3024
1827/3024
1828/3024
1829/3024
1830/3024
1831/3024
1832/3024
1833/3024
1834/3024
1835/3024
1836/3024
1837/3024
1838/3024
1839/3024
1840/3024
1841/3024
1842/3024
1843/3024
1844/3024
1845/3024
1846/3024
1847/3024
1848/3024
1849/3024
1850/3024


2571/3024
2572/3024
2573/3024
2574/3024
2575/3024
2576/3024
2577/3024
2578/3024
2579/3024
2580/3024
2581/3024
2582/3024
2583/3024
2584/3024
2585/3024
2586/3024
2587/3024
2588/3024
2589/3024
2590/3024
2591/3024
2592/3024
2593/3024
2594/3024
2595/3024
2596/3024
2597/3024
2598/3024
2599/3024
2600/3024
2601/3024
2602/3024
2603/3024
2604/3024
2605/3024
2606/3024
2607/3024
2608/3024
2609/3024
2610/3024
2611/3024
2612/3024
2613/3024
2614/3024
2615/3024
2616/3024
2617/3024
2618/3024
2619/3024
2620/3024
2621/3024
2622/3024
2623/3024
2624/3024
2625/3024
2626/3024
2627/3024
2628/3024
2629/3024
2630/3024
2631/3024
2632/3024
2633/3024
2634/3024
2635/3024
2636/3024
2637/3024
2638/3024
2639/3024
2640/3024
2641/3024
2642/3024
2643/3024
2644/3024
2645/3024
2646/3024
2647/3024
2648/3024
2649/3024
2650/3024
2651/3024
2652/3024
2653/3024
2654/3024
2655/3024
2656/3024
2657/3024
2658/3024
2659/3024
2660/3024
2661/3024
2662/3024
2663/3024
2664/3024
2665/3024
2666/3024
2667/3024
2668/3024
2669/3024
2670/3024
